In [ ]:
%%writefile main.nf
#!/usr/bin/env nextflow

/* pipeline input parameters */
dicom_data = "$baseDir/sdk_data/06/8654bc98ec4951b25adbe8a0dde5dc/test_few/*.dcm"

process run_inference {

    memory '5 GB'
    accelerator 1
    
    label 'inference'
    publishDir "$baseDir/sdk_data"
    
    input:
    path dicom_files
    
    output:
    stdout emit: inference_log
    path('*.csv'), emit: csv_files
    
    script:
    """
    python /app/inference_modified_for_batch.py --dicom-input $dicom_files --classification-out-csv ${dicom_files}.classification_results.csv  
    """

}

// Define the entry workflow (initial workflow for Nextflow to run)
workflow {
    dicom_files = Channel.fromPath(dicom_data)
    run_inference(dicom_files)
}


In [ ]:
%%writefile nextflow.config

process {
    withLabel: inference {
        executor = 'awsbatch'
        queue = 'placeholder'
        container = 'placeholder'
        containerOptions = "--gpus all"
    }
}

aws {
	batch {
		cliPath = '/home/ec2-user/miniconda/bin/aws'
		jobRole = 'placeholder'
	}
}

workDir = 'placeholder'

docker.enabled = true

In [ ]:
!nextflow run main.nf -dsl2